In [1]:
import picnik as pnk           #path: Materiales/Pyrolysis \Kinetics/PyRolysis
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from scipy.optimize import minimize_scalar as mini
from scipy.integrate import romberg 
%matplotlib tk

In [10]:
#NP_TG
f = ['examples/final_NP/NP_050/NP_050.csv',
    'examples/final_NP/NP_075/NP_075.csv',
    'examples/final_NP/NP_100/NP_100.csv',
    'examples/final_NP/NP_120/NP_120.csv']

In [14]:
#NP_SCO_H
f = ['examples/Magneto/H/B/NP_025_hB.csv',
     'examples/Magneto/H/B/NP_050_hB.csv',
     'examples/Magneto/H/B/NP_200_hB.csv']

In [24]:
#NP_SCO_C
f = ['examples/Magneto/C/B/NP_025_cB.csv',
     'examples/Magneto/C/B/NP_050_cB.csv',
     'examples/Magneto/C/B/NP_200_cB.csv']

In [2]:
#TCN_TG
f = ['examples/final_TCN/_050/TCN_050.csv',
     'examples/final_TCN/_075/TCN_075.csv',
     'examples/final_TCN/_100/TCN_100.csv',
     'examples/final_TCN/_120/TCN_120.csv']

In [198]:
#TCN_SCO_H
f = ['examples/exp-H/H_040.csv',
     'examples/exp-H/H_060.csv',
     'examples/exp-H/H_100.csv']

In [102]:
#TCN_SCO_C
f = ['examples/exp-C/C_040.csv',
     'examples/exp-C/C_060.csv',
     'examples/exp-C/C_100.csv']

In [3]:
xtr   = pnk.DataExtraction()
B, T0 = xtr.read_files(f)
xtr.Conversion(350,590)
T_DF, t_DF, dadt_DF, aT_DF, at_DF = xtr.Isoconversion(advanced = True, method='step', d_a = 0.005)

Files to be used: 
['examples/final_TCN/_050/TCN_050.csv', 'examples/final_TCN/_075/TCN_075.csv', 'examples/final_TCN/_100/TCN_100.csv', 'examples/final_TCN/_120/TCN_120.csv']
 
Reading files and creating DataFrames...

The computed heating rates are:

4.95 K/min
7.44 K/min
9.92 K/min
12.00 K/min
The temperature range was set to (350.0,590.0) K
Computing conversion values...
Done
Creating Isoconversion DataFrames...
Done


In [4]:
Ea = pnk.ActivationEnergy(B, T0, TempIsoDF = T_DF, TempAdvIsoDF=aT_DF, timeAdvIsoDF=at_DF)

In [5]:
aVy = Ea.aVy((1,110),N=1,p=0.90,method='trapezoid')

Advanced Vyazovkin method: Computing activation energies...


/home/erock/Materiales/Pyrolysis Kinetics/PyRolysis/integration.py:81: RuntimeWarning: divide by zero encountered in double_scalars
  hp = (b-a)/Nprime
/home/erock/Materiales/Pyrolysis Kinetics/PyRolysis/integration.py:88: RuntimeWarning: invalid value encountered in double_scalars
  Icp = I0p +(hp*Fcp)


Done.


In [6]:
#plt.errorbar(at_DF.index.values[25:-5:20],
#             aVy_fit[0][::20],
#             aVy_fit[1][::20], 
#             marker='s',
#             elinewidth=4,
#             alpha=0.7,
#             label= 'Romberg')

plt.errorbar(at_DF.index.values[1::3],
             aVy[0][::3],
             aVy[1][::3], 
             marker='s',
             elinewidth=3,
             alpha=0.7)
#             label= 'Simpson')

#plt.errorbar(at_DF.index.values[25:-25:20],
#             aVy_t[0][::20],
#             aVy_t[1][::20], 
#             marker='s',
#             elinewidth=2,
#             alpha=0.7,
#             label= 'Trapecio')
plt.legend()
plt.xlabel(r'$\alpha$')
plt.ylabel(r'$E_{\alpha}$')
plt.xlim(0.05,0.95)

(0.05, 0.95)

In [225]:
Ea.export_Ea(E_aVy=True, N=6, file_t="csv" )

Exporting activation energies...


Input file name for the adv.Vy results: TCN_SCOh_aVy.


Results exported
Done.


## Non.isothermal prediction

In [6]:
ap,Tp,tp = Ea.prediction(aVy[0],B=7.5,N=1,bounds = (5,5))

In [7]:
plt.plot(tp,ap,'.')
plt.plot(xtr.t[1],xtr.alpha[1])

## Isothermal (lifetime) Prediction

In [38]:
ap,Tp,tp = Ea.prediction(aVy[0],B=0,isoT = (320), alpha=0.95,N=1,bounds = (50000,50000))
#ap1,Tp1,tp1 = Ea.prediction(aVy[0],B=0,isoT = 420, alpha=0.25,N=1,bounds = (0.15,0.15))
#ap2,Tp2,tp2 = Ea.prediction(aVy[0],B=0,isoT = 420, alpha=0.25,N=1,bounds = (500000,500000))

In [39]:
plt.plot(tp,ap,'.',alpha=0.5,label='5')
#plt.plot(tp1,ap1,'^--',alpha=0.5,label='0.15')
#plt.plot(tp2,ap2,'-',alpha=0.5,label='500000')
plt.xlabel('time [min]')
plt.ylabel(r'conversion ($\alpha$)')

plt.legend()

In [11]:
at_DF.iloc[0].values[0:len(B)]

array([7.3923, 4.4471, 3.3924, 3.2798])